<a href="https://colab.research.google.com/github/EntropyP/News_Update/blob/main/News_Update_Main_Program_UserVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Update acquisition : Main Program

In [1]:
! pip install newsapi-python

# import libraries
from newsapi import NewsApiClient
import datetime as datetime
import pandas as pd

# input api_key (Request from https://newsapi.org/)
newsapi = NewsApiClient(api_key = 'a51013f42c6d46e3b3fa974d7363d177')

#Input key search and maximum page search
key_search = input('Please input your key search: ')
page_search = 100

#Create final dataframe which will collect all news in the focus area according to key_search
df_final = pd.DataFrame()

#Search data from each key_search
for key in key_search:
  try:
    data = newsapi.get_everything(q = key, language = 'en', page_size= page_search, sort_by='relevancy')
    articles = data['articles']
    df_news = pd.DataFrame(articles)
    df_news['key'] = key
    df_final = df_final.append(df_news,ignore_index=True)
  except:
    print('There is no search result for "', key ,'"')

#The final dataframe which was classified by topic
df_final['source_name'] = df_final['source'].apply(pd.Series)['name']
df_final.head()

#Save to excel
file_name = input('For save, Please enter file name: ')
df_final.to_excel(file_name + '.xlsx')

#Scrapping
import bs4
import requests

headers = {
    'User-agent':
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

#reuters
def reuters():
  content = soup.findAll('p', class_= 'Text__text___3eVx1j Text__dark-grey___AS2I_p Text__regular___Bh17t- Text__large___1i0u1F Body__base___25kqPt Body__large_body___3g04wK ArticleBody__element___3UrnEs')
  content_final = ''
  for con in content:
    content_final += con.text
  return content_final

#yahoo
def yahoo():
  content = soup.findAll('p')
  content_final = ''
  for con in content:
    if '<p class=' in str(con):
      continue
    else:
      content_final += con.text
  return content_final

#summary library
from gensim.summarization.summarizer import summarize

#create dataframe for data storage
df_extract = df_final.copy(deep=True)
df_extract = df_extract[['title','url','publishedAt','key','source_name']]
df_extract['Summary'] = None
df_extract['Full_Text'] = None

#main program
for row, column in df_extract.iterrows():
  if column['source_name'] == 'Reuters':
    ori_link = column['url']
    link = requests.get(ori_link,headers=headers)
    soup = bs4.BeautifulSoup(link.text,'lxml')
    try:
      text = reuters()
      df_extract.loc[row, ['Summary']] = summarize(text, ratio=0.3)
      df_extract.loc[row, ['Full_Text']] = text
    except:
      pass
  elif column['source_name'] == 'Yahoo Entertainment':
    ori_link = column['url']
    link = requests.get(ori_link,headers=headers)
    soup = bs4.BeautifulSoup(link.text,'lxml')
    try:
      text = yahoo()
      df_extract.loc[row, ['Summary']] = summarize(text, ratio=0.3)
      df_extract.loc[row, ['Full_Text']] = text
    except:
      pass

#Save to excel
df_extract = df_extract.dropna()
df_extract.to_excel(file_name + '_summary.xlsx')
df_extract.head()

Please input your key search: SCG
For save, Please enter file name: SCG_file


,title,url,publishedAt,key,source_name,Summary,Full_Text
16,2021's cutest baby animals - Reuters,https://www.reuters.com/video/watch/idOVF9YKDWN,2021-12-28T11:15:00Z,S,Reuters,,
17,2021's major cryptocurrency trends - Reuters.com,https://www.reuters.com/video/watch/idOVFA8JW4N,2021-12-30T18:13:43Z,S,Reuters,,
21,LIVE MARKETS 2021's swagger signals 2022 stren...,https://www.reuters.com/markets/asia/live-mark...,2022-01-04T15:45:00Z,S,Reuters,"In past occasions, the index has risen in the ...",Jan 4 - Welcome to the home for real-time cove...
22,Floods kill ten in S. African city - Reuters,https://www.reuters.com/video/watch/idOVFTNRTDF,2022-01-10T17:48:17Z,S,Reuters,,
23,Tesco and M&S lift outlook after strong Christ...,https://www.reuters.com/video/watch/idOVFU2PDVV,2022-01-13T11:32:54Z,S,Reuters,,


In [2]:
df_final.head()

,source,author,title,description,url,urlToImage,publishedAt,content,key,source_name
0,"{'id': 'engadget', 'name': 'Engadget'}",Cherlynn Low,Samsung will unveil its next Galaxy S flagship...,After introducing S Pen support to the Galaxy ...,https://www.engadget.com/samsung-galaxy-s22-no...,https://s.yimg.com/os/creatr-uploaded-images/2...,2022-01-20T23:00:36Z,After introducing S Pen support to the Galaxy ...,S,Engadget
1,"{'id': 'engadget', 'name': 'Engadget'}",Jon Fingas,Watch a drag race between Tesla's Model S Plai...,"Now that the Lucid Air is a practical reality,...",https://www.engadget.com/tesla-model-s-plaid-l...,https://s.yimg.com/os/creatr-uploaded-images/2...,2022-01-25T19:50:13Z,"Now that the Lucid Air is a practical reality,...",S,Engadget
2,"{'id': 'engadget', 'name': 'Engadget'}",Kris Holt,Microsoft is no longer making Xbox One consoles,Microsoft and Sony are struggling to keep up w...,https://www.engadget.com/microsoft-xbox-one-di...,https://s.yimg.com/os/creatr-uploaded-images/2...,2022-01-13T15:21:22Z,Microsoft and Sony are struggling to keep up w...,S,Engadget
3,"{'id': None, 'name': 'New York Times'}",Lauren McCarthy,"S&P 500, Burkina Faso, Hippos: Your Monday Eve...",Here’s what you need to know at the end of the...,https://www.nytimes.com/2022/01/24/briefing/sp...,https://static01.nyt.com/images/2022/01/24/mul...,2022-01-24T22:58:15Z,9. An ode to the original veggie burger.\r\nIn...,S,New York Times
4,"{'id': None, 'name': 'New York Times'}",Emmanuel Morgan,"Francis Ngannou is the U.F.C.'s Fearsome, Quie...","Francis Ngannou, known widely for his devastat...",https://www.nytimes.com/2022/01/21/sports/fran...,https://static01.nyt.com/images/2022/01/22/spo...,2022-01-21T20:28:13Z,"Dana White, the U.F.C. president, said that be...",S,New York Times
